2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)?

Логично было бы предположить, что имеются ввиду посылки, которые должны были быть доставлены в определенный месяц, который  определяет обещанная дата доставки (order_estimated_delivery_date). Однако в одной из таблиц видно, 
что order_estimated_delivery_date есть даже тогда, когда нет order_approved_at. 
Т.к. условие неконкретно, будем считать, что order_estimated_delivery_date появляется после размещения заказа, т.е. есть order_purchase_timestamp.

При этом могут быть варианты отсутствия доставки:

  1. Заказ опоздал и приехал в следующий месяц, соответственно в обещаный месяц он не доставился. Причина будет называться: 
    1.1. Доставлен с опазданием (delivered_late). 
Это значит месяц в колонке order_delivered_customer_date должен быть больше, чем в order_estimated_delivery_date. 
(Вариант, когда заказ прехал на месяц раньше, рассматривать не будем, т.к. в этом случае проблемы нет, все хорошо).
  2. Заказ вообще не приехал и отменился на какой-то стадии. Причины: 
    2.1. Не оплачен (not_paid), т.е. нет order_approved_at.
    2.2. Оплачен, но нет order_delivered_carrier_date, т.е. не передан в доставку (not_sent_for_delivery).
    2.3. Оплачен и передан в доставку, но нет order_delivered_customer_date , т.е. не доставлен покупателю. 

Назовем это: задержан службой доставки (delayed_by_delivery_service).
Собственно эти 4 колонки и будем строить.

Есть еще shipping_limit_date —  'максимальная дата доставки продавцом для передачи заказа партнеру по логистике' в таблице olist_order_items_dataset.csv, и можно было бы сравнивать с ней фактическую дату передачи в доставку 
(order_delivered_carrier_date), но в условии не сказано, зачем эта дата, и что будет, если продавец не успеет,
поэтому будем придерживаться приведенного выше плана.

In [1]:
# Блок импорта библиотек
import pandas as pd
import requests
from urllib.parse import urlencode
import numpy as np
import plotly.express as px
from datetime import timedelta
import matplotlib.pyplot as plt
from datetime import datetime

Получим прямые ссылки на все датафреймы

olist_customers_datase.csv — таблица с уникальными идентификаторами пользователей

olist_orders_dataset.csv — таблица заказов

olist_order_items_dataset.csv — товарные позиции, входящие в заказы


In [2]:
# olist_customers_datase.csv
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key_1 = 'https://disk.yandex.ru/d/FUi5uSd6BfG_ig' 
# Получаем загрузочную ссылку
final_url_1 = base_url + urlencode(dict(public_key=public_key_1))
response = requests.get(final_url_1)
download_url_1 = response.json()['href']
olist_customers_dataset = pd.read_csv(download_url_1, sep=',')

# olist_orders_dataset.csv
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key_2 = 'https://disk.yandex.ru/d/t9Li4JOfgxuUrg' 
# Получаем загрузочную ссылку
final_url_2 = base_url + urlencode(dict(public_key=public_key_2))
response = requests.get(final_url_2)
download_url_2 = response.json()['href']
olist_orders_dataset = pd.read_csv(download_url_2, sep=',')

# olist_order_items_dataset.csv
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key_3 = 'https://disk.yandex.ru/d/Gbt-yAcQrOe3Pw' 
# Получаем загрузочную ссылку
final_url_3 = base_url + urlencode(dict(public_key=public_key_3))
response = requests.get(final_url_3)
download_url_3 = response.json()['href']
olist_order_items_dataset = pd.read_csv(download_url_3, sep=',')

In [3]:
# посмотрим на типы данных

olist_orders_dataset.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [4]:
# переведем нужные столбцы в даты

olist_orders_dataset['order_purchase_timestamp'] = (
    pd.to_datetime(olist_orders_dataset['order_purchase_timestamp']))

olist_orders_dataset['order_estimated_delivery_date'] = (
    pd.to_datetime(olist_orders_dataset['order_estimated_delivery_date']))

In [5]:
# проверим

olist_orders_dataset.dtypes

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                        object
order_delivered_carrier_date             object
order_delivered_customer_date            object
order_estimated_delivery_date    datetime64[ns]
dtype: object

In [6]:
# посчитаем разницу между временем размещения заказа и обещаным временем доставки.

olist_orders_dataset['diff_1'] = (
    (olist_orders_dataset.order_estimated_delivery_date -\
     olist_orders_dataset.order_purchase_timestamp).astype('timedelta64[D]'))

In [7]:
olist_orders_dataset.head(10)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,diff_1
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,15.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,19.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,26.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,26.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,12.0
5,a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered,2017-07-09 21:57:05,2017-07-09 22:10:13,2017-07-11 14:58:04,2017-07-26 10:57:55,2017-08-01,22.0
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaN,NaN,2017-05-09,27.0
7,6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,delivered,2017-05-16 13:10:30,2017-05-16 13:22:11,2017-05-22 10:07:46,2017-05-26 12:55:51,2017-06-07,21.0
8,76c6e866289321a7c93b82b54852dc33,f54a9f0e6b351c431402b8461ea51999,delivered,2017-01-23 18:29:09,2017-01-25 02:50:47,2017-01-26 14:16:31,2017-02-02 14:08:10,2017-03-06,41.0
9,e69bfb5eb88e0ed6a785585b27e16dbf,31ad1d1b63eb9962463f764d4e6e0c9d,delivered,2017-07-29 11:55:02,2017-07-29 12:05:32,2017-08-10 19:45:24,2017-08-16 17:14:30,2017-08-23,24.0


По разнице diff_1 видно, что обещаный срок доставки не фиксированный, как это обычно у продавцов, а какое-то случайное
количество дней. Это странно, ну да ладно.

In [9]:
# Перекачаем датафрейм olist_orders_dataset.csv, т.к. выше вносились изменения
# и поработаем с ним без изменения типов данных колонок

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key_2 = 'https://disk.yandex.ru/d/t9Li4JOfgxuUrg' 
# Получаем загрузочную ссылку
final_url_2 = base_url + urlencode(dict(public_key=public_key_2))
response = requests.get(final_url_2)
download_url_2 = response.json()['href']
olist_orders_dataset = pd.read_csv(download_url_2, sep=',')

In [10]:
olist_orders_dataset.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [11]:
# Приведем нужные колонки с датами к формату "год-месяц"

olist_orders_dataset['order_estimated_delivery_date'] = (
    olist_orders_dataset['order_estimated_delivery_date'].str.split('-').str[0]+\
    '-'+ olist_orders_dataset['order_estimated_delivery_date'].str.split('-').str[1])

olist_orders_dataset['order_delivered_customer_date'] = (
    olist_orders_dataset['order_delivered_customer_date'].str.split('-').str[0]+\
    '-'+ olist_orders_dataset['order_delivered_customer_date'].str.split('-').str[1])

In [12]:
olist_orders_dataset.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10,2017-10
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08,2018-08
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08,2018-09
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12,2017-12
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02,2018-02


In [13]:
olist_orders_dataset.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [14]:
# добавим колонку diff_2 в которой будет разница между 
# месяцем обещаной доставки и месяцем фактической доставки

olist_orders_dataset['diff_2'] = (pd.to_datetime(olist_orders_dataset.order_estimated_delivery_date)\
                                - pd.to_datetime(olist_orders_dataset.order_delivered_customer_date)) /\
 np.timedelta64(1, 'D')
                                   

In [15]:
olist_orders_dataset.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,diff_2
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10,2017-10,0.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08,2018-08,0.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08,2018-09,31.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12,2017-12,0.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02,2018-02,0.0


In [16]:
olist_orders_dataset.dtypes

order_id                          object
customer_id                       object
order_status                      object
order_purchase_timestamp          object
order_approved_at                 object
order_delivered_carrier_date      object
order_delivered_customer_date     object
order_estimated_delivery_date     object
diff_2                           float64
dtype: object

In [25]:
# если есть опоздание, т.е. причина а1 - delivered_late, то эта разница будет отрицательной. 
# посмотрим, есть ли такие случаи

olist_orders_dataset.query('diff_2 < 0').order_id.count()

1980

видим, что работает, случаи опоздания есть.
Создадим датафрейм, где просто посчитаем общее количество причин 1.1, 2.1-2.3

In [19]:
olist_orders_dataset = olist_orders_dataset.fillna(0)

In [20]:
not_delivered = pd.DataFrame([[olist_orders_dataset.query('diff_2 < 0').order_id.count(),
                             olist_orders_dataset.query('order_approved_at == 0').order_id.count(),
                             olist_orders_dataset.query('order_delivered_carrier_date == 0\
                                                         & order_approved_at != 0').order_id.count(),
                             olist_orders_dataset.query('order_delivered_customer_date == 0\
                                                         & order_delivered_carrier_date != 0\
                                                         & order_approved_at != 0').order_id.count()]],
columns=['delivered_late', 'not_paid', 'not_sent_for_delivery', 'delayed_by_delivery_service'])

In [21]:
not_delivered

,delivered_late,not_paid,not_sent_for_delivery,delayed_by_delivery_service
0,1980,160,1637,1183


Теперь построим датафрейм, отвечающий на вопрос задачи:
сколько заказов в месяц в среднем не доставляется и по каким причинам.

In [23]:
not_delivered_mean = pd.DataFrame([[olist_orders_dataset.query('diff_2 < 0') \
                                    .groupby('order_estimated_delivery_date', as_index = False) \
                                    .agg({'order_id' : 'count'}) \
                                    .order_id.mean(),
                                    
                                    olist_orders_dataset.query('order_approved_at == 0') \
                                     .groupby('order_estimated_delivery_date', as_index = False) \
                                     .agg({'order_id' : 'count'}) \
                                     .order_id.mean(),
                                    
                                    olist_orders_dataset.query('order_delivered_carrier_date == 0\
                                                                  & order_approved_at != 0') \
                                     .groupby('order_estimated_delivery_date', as_index = False) \
                                     .agg({'order_id' : 'count'}) \
                                     .order_id.mean(),
                                    
                                    olist_orders_dataset.query('order_delivered_customer_date == 0\
                                                                & order_delivered_carrier_date != 0\
                                                                & order_approved_at != 0') \
                                     .groupby('order_estimated_delivery_date', as_index = False) \
                                     .agg({'order_id' : 'count'}) \
                                     .order_id.mean()]],
columns=['delivered_late', 'not_paid', 'not_sent_for_delivery', 'delayed_by_delivery_service'])

In [24]:
not_delivered_mean

,delivered_late,not_paid,not_sent_for_delivery,delayed_by_delivery_service
0,94.285714,7.272727,71.173913,51.434783


Ответ приведен в датафрейме not_delivered_mean. Расшифровка причин: 

delivered_late - доставлен с опазданием (не в месяц обещаной доставки),

not_paid - заказ не оплачен,

not_sent_for_delivery - не передан в доставку продавцом,

delayed_by_delivery_service - задержка в службе доставки.